# Database parsing

In [1]:
import pandas, numpy as np
import parsingDatabaseUtils, re
import xml, itertools, collections, xml.etree.ElementTree as ET
import tqdm, importlib, dateparser, dateparser.search

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)

procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)
registrosByCaso = registros.groupby('Caso')

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,27,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
classificationProcedures = {'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}

## Select a case
Identify all  cases with an associated procedure

In [4]:
# Identify all  cases with an associated procedure

In [5]:
interventionToCase = {}
caseToProcedureBirths = {}
checkSeveralProcedures = []
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcionProcedimiento = et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo')
        interventionToCase[idDescripcionProcedimiento] = (r.Caso, r.NumeroHistoria, i)
        procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion)[0]
        if classificationProcedures[procedureType] == 'p':
            
            #If it is the case that they need to stop labour because a c-section is needed
            if '<postDiagnosticoPrincipal>O821 - PARTO POR CESAREA DE EMERGENCIA</postDiagnosticoPrincipal>' in \
                procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion:
                continue
                
            elif r.Caso in caseToProcedureBirths:
                r1 = caseToProcedureBirths[r.Caso]
                r2 =procedimientos.loc[idDescripcionProcedimiento]
                if r1.XmlDescripcion == r2.XmlDescripcion:
                    continue
                else:
                    checkSeveralProcedures.append(r.Caso)
                    print('error', r.Caso)
                    if r1.FechaRegistro > r2.FechaRegistro:
                        caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]
            else:
                caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]

error AD350190
error AD351571
error AD350755
error AD353436
error AD353902
error AD354284
error AD359075
error NN
error NN
error AD366060
error AD363741
error AD363741



In [6]:
importlib.reload(parsingDatabaseUtils)

<module 'parsingDatabaseUtils' from '/Users/gbernardino/toolsCMRC/parsingDatabaseUtils.py'>

In [7]:
import birthDatasetStructure
importlib.reload(birthDatasetStructure)
# Split in newborn
registrosByCaso = registros.groupby('Caso')
processedDatasets = {}
for c, p in tqdm.tqdm_notebook(caseToProcedureBirths.items()):
    if c not in casos.index:
        print('ERROR!', c)
        continue
    processedDatasets[c] = birthDatasetStructure.BirthDataset(c, casos.loc[c], p, registrosByCaso.get_group(c), pacientes)


ERROR! NN



In [8]:
for c, p in processedDatasets.items():
    
    if len(p.registrosRecienNacido) > 1:
        print(c, p.procTypeId,  len(p.registrosRecienNacido))


In [9]:
importlib.reload(parsingDatabaseUtils)
resSIP = {}
breakLoop = False
count = 0
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    resSIP[c] = parsingDatabaseUtils.getMotherData(p)
    resSIP[c].update(parsingDatabaseUtils.getInformationFromProcedureDescription(p))
    for k in p.registrosRecienNacido:
        resSIP[c].update(parsingDatabaseUtils.getNewbornData(p, k))
df = pandas.DataFrame.from_dict(resSIP, orient = 'index')
#l = list(df[(df.VAR_0294 != df.VAR_0294) & (df.VAR_0293 != df.VAR_0293)].index)
#print(len(l))
print(count)


0


In [10]:
df.to_csv('resultsSIP.csv')

In [11]:
for c,  _ in df[df['sufrimientoFetal'] == 'SI'].iterrows():
    et = ET.fromstring(processedDatasets[c].epicrisis.RegistroXML)
    print(parsingDatabaseUtils.findInXML('MedicamentosAdministrado', et ))


Fecha: 04/12/2017
-19972965-01 - SOLUCION SALINA 0.9 %- Cantidad: 2
-201128 - Jeringa Desechable 1 ml x 27 G/ 1/2&amp;quot; - Cantidad: 2
-51076-02 - SOLUCION SALINA 0.9 % - Cantidad: 1
-IM5034 - MORFINA CLORHIDRATO - Cantidad: 1
-IM5059 - BUPIVACAINA+DEXTROSA (BUPIROP PESADO) - Cantidad: 1
-IM5064 - LACTATO DE RINGER - Cantidad: 6
-IM5065 - LACTATO DE RINGER - Cantidad: 1
-IM5093 - RECIPIENTE DE SUCCION FLEXIBLE (LINER) de 3 LT - Cantidad: 1
-INS001 - ABOCATH NO. 18 - Cantidad: 1
-INS007 - EQUIPO MACROGOTEO - Cantidad: 1
-INS016 - SONDA NELATON # 16 - Cantidad: 1
-INS039 - VICRYL 1 CT-1(HR37S) x 90 CM - Cantidad: 1
-INS042 - AGUJA ESPINAL TIPO QUINCKE # 27 X 3 1/2 x 90 MM - Cantidad: 1
-INS089 - NYLON 3/0 (CDSS24) AGUJA CURVA - Cantidad: 1
-INS1019 - CATGUT 0 CT/1 ( HR37S) x 70 CM - Cantidad: 1
-INS1020 - CATGUT 1 CT/1 (HR37S) x 90 CM - Cantidad: 1
-INS121 - SONDA NELATON # 8 - Cantidad: 1
-INS127 - LAPIZ DE ELECTROBISTURI - Cantidad: 1
-INS225 - Jeringa Desechable 10 ml x 21 G/1 1/2

In [12]:
df[df['VAR_0425'].isna()]

,VAR_0006,VAR_0012,VAR_0013,VAR_0015,VAR_0018,VAR_0019,VAR_0020,VAR_0022,VAR_0024,VAR_0026,...,sufrimientoFetal,VAR_0425,VAR_0343,VAR_202,VAR_0060,VAR_0294,VAR_0335,VAR_0011,VAR_0381,VAR_0330
AD215646,1987-11-02 00:00:00,B,C,C,CMRC,VEN18495140,A,B,B,A,...,NO,NaN,A,A,NaN,1,NaN,NaN,NaN,NaN
AD239256,1998-10-24 00:00:00,NaN,NaN,D,CMRC,VEN26697250,A,A,B,A,...,NO,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AD254941,1998-09-23 00:00:00,B,C,D,CMRC,VEN26575169,A,B,A,A,...,NO,NaN,B,NaN,NaN,1,NaN,NaN,NaN,NaN
AD273164,1997-10-31 00:00:00,B,B,D,CMRC,VEN28391456,A,A,A,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AD275005,2001-11-17 00:00:00,B,C,D,CMRC,VEN32337201,A,A,A,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AD367709,2001-01-22 00:00:00,B,C,D,CMRC,VEN27742994,A,A,A,A,...,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN
AD368225,1999-06-14 00:00:00,B,C,D,CMRC,VEN27059849,A,A,A,A,...,NO APLICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AD368521,2000-04-22 00:00:00,B,C,D,CMRC,VEN27039472,A,A,B,A,...,NO,NaN,A,A,NaN,0,NaN,NaN,NaN,NaN
AD368707,2002-04-18 00:00:00,B,C,C,CMRC,VEN32461184,A,A,A,A,...,NO,NaN,A,A,NaN,1,NaN,NaN,NaN,NaN


In [13]:
for _, p in processedDatasets['AD218865'].registrosRecienNacido['796790'].items():
    parsingDatabaseUtils.prettyPrintXML(p.RegistroXML)
    print('------')

<?xml version="1.0" ?>
<C Asunto="EVOLUCION" Caso="AD218865" CentroA="01" Diagnostico="NULL" IdPaciente="164688" Modulo="01" Padre="796790" PlanAdm="100" Prestador="73086292" Raiz="165" Registro="166">
	<row NombreCampo="CodigoPlanAdministradora" NombreTabla="CamposTexto" ValorCampo="100|0001"/>
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&lt;B&gt;
&lt;P align=center&gt;EVOLUCI&#211;N&amp;nbsp; &lt;/P&gt;&lt;/B&gt;
&lt;P align=center&gt;　&lt;B&gt;RN CON DIAGNOSTICO DE: &lt;/B&gt;&lt;BR&gt;1. RNTAEG (29/03/2017&amp;nbsp;10:25PM)&lt;BR&gt;2. RIESGO DE SEPSIS POR NO CONTROL PRENATAL&lt;BR&gt;&lt;STRONG&gt;PESO:&lt;/STRONG&gt;&amp;nbsp;3210&amp;nbsp; gm&lt;STRONG&gt;TALLA: 50 cm, PC: 32 PT: 33&lt;/STRONG&gt;&lt;/P&gt;
&lt;P align=justify&gt;&lt;STRONG&gt;GRUPO SANGUINEO&amp;nbsp;DE MADRE: A+ &lt;/STRONG&gt;&lt;/P&gt;
&lt;P align=justify&gt;&lt;STRONG&gt;SUBJETIVO:&lt;/STRONG&gt; MADRE REFIERE QUE EL PACIENTE PASO BUENA NOCHE, TOLERANDO VIA ORAL, NIEGA FIEBRE, N

In [14]:
len(df.loc[df['VAR_0314'] != df['VAR_0314']])

7

In [15]:
errorCount = 0
for c,v in resSIP.items():
    if len(v) == 0:
        errorCount += 1
print(errorCount)
        

0


In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
def f(c):
    parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].procedure.XmlDescripcion)
interact(lambda i: f(l[i]), i = (0, len(l)))

NameError: name 'l' is not defined

In [20]:
noEpicrsis = []
noIngreso = []

noDischarge = []
for c, p in processedDatasets.items():
    if p.epicrisis is None:
        noDischarge.append(c)
    if p.ingreso is None:
        noIngreso.append(c)
print('no emergency' , len(noIngreso), 'no epi', len(noDischarge), len(processedDatasets))

no emergency 81 no epi 12 1929


In [35]:
c = noIngreso[10]
registrosByCaso.get_group(c)
#parsingDatabaseUtils.prettyPrintXML(registros.loc['1025813', 'RegistroXML'])
#casos.loc[casos.Paciente == 184931 ]

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1226816,204744,AD335514,145,NaN,2019-04-22 08:55:02.790,NaN,NaN,MBANQUEZ,1128057935,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1227016,204744,AD335514,94,Notas de Ingreso a Piso,2019-04-22 12:38:48.560,NaN,NaN,HARGOZ7,73086292,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-22 12:37:00.000,NaN,False,NaN
1226853,204744,AD335514,165,Registro del recién nacido,2019-04-22 09:35:34.513,NaN,Padre,sa,1140866516,100,...,False,NaN,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN
1226980,204744,AD335514,145,NaN,2019-04-22 11:50:40.593,NaN,NaN,MBANQUEZ,1128057935,1122018,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1227436,204744,AD335514,106,Evoluci&#243;n,2019-04-23 09:41:14.590,94.0,1227016,JRAMIREZ,9068204,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-23 09:40:00.000,NaN,False,NaN
1227561,204744,AD335514,166,EVOLUCION DEL RECIEN NACIDO,2019-04-23 12:02:16.000,165.0,1226853,HARGOZ7,73086292,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-23 11:31:00.000,NaN,False,NaN
1227997,204744,AD335514,106,REPORTE DE PARACLINICOS,2019-04-24 06:48:29.147,94.0,1227016,ADVALLE,73543467,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-24 06:48:00.000,NaN,False,NaN
1228099,204744,AD335514,106,Evoluci&#243;n,2019-04-24 10:03:02.170,94.0,1227016,JRAMIREZ,9068204,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-24 09:57:00.000,NaN,False,NaN
1228110,204744,AD335514,450,Epicrisis de la Historia Notas de Ingreso a Piso,2019-04-24 10:09:38.977,1.0,AD335514,JRAMIREZ,9068204,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN


In [18]:
count = 0
for c, p in processedDatasets.items():
    if p.epicrisis is None:
        continue
    et = ET.fromstring(p.epicrisis.RegistroXML)
    ant = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', et))
    if not 'paracli' in ant:
        count += 1
    else:
        print(ant)

personales 
 7 > menarquia 15
vida sexual 27 
vida obstetrica 28
g1p0a0c0
fum 15/01/16 hoy embarazo de 39.6 semanas 
companeros sexuales 1
controles prenatales 3 (pobremente controlado)
tipo de sangre o+
planificacion niega 

ecografias 
08/junio/16 embarazo de 20 3 semanas para hoy embarazo de 39,4 semanas
27/junio/16 embarazo de 23 semanas para hoy embarazon de 41 semanas
22/agosto/16 embarazo de 32 semanas para hoy embarzo de 40,3 semanas

paraclinicos 
08/junio/16 hemograma hb 10,4 hto 31 leu 7,700
27/junio/16 antitoxoplasma ( )
no trae vih ni vdrl


 5 > niega
 familiares 
 niega
 alergicos 
 
 quirurgicos
personales 
 1 > asma 
 
 7 > g 1 p 0 a 0 c 0 
fum no recuerda 
menarquia 11 
vida marital 15 
vida obstetrica 18 
companerops sexuales 1 
tipo de sangre o+ 
 
ecografias 
6/6/2016 embarazo de 21 semanas hoy 34.3 
 
paraclinicos 
12 de julio del 2016 hemograma hb 10.9gr/dl htc 34.5% plq 283.000 leu 10.100 
 familiares 
 niega 
 alergicos 
 
 quirurgicos
personales 
 2 > litotrip

In [ ]:
count

In [ ]:
c = 'AD284225'
k = next(iter(processedDatasets[c].registrosRecienNacido.keys()))
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].registrosRecienNacido[k][k].RegistroXML)


In [62]:
count

249

In [61]:
c = list(processedDatasets.keys())[884]
count = 0
for c in processedDatasets.keys():
    try:
        if 'ECLAMPS' in processedDatasets[c].epicrisis.RegistroXML:
            count += 1
            #parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)
    except:
        pass

In [151]:
df.to_csv('test.csv')

In [90]:
c = list(processedDatasets.keys())[244]
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD336169" CentroA="01" CodigoDiagnosticoRelacionado1="Z348" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O623" IdPaciente="206122" Modulo="01" Padre="AD336169" PlanAdm="100" Prestador="1128057935" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  1-&amp;gt;  ASMA

  2-&amp;gt; NIEGA

  5-&amp;gt;  NIEGA

  7-&amp;gt; 
MENARQUIA: 8 A&#209;OS 
INICIO DE VIDA SEXUAL: 15 A&#209;OS
INICIO DE VIDA OBSTETRICA: 15 A&#209;OS 
G1 P0 C0 A0
PERIODOS MENSTRUALES:  REGULARES
FUM:  NO SABE 
PLANIFICACION FAMILIAR: NIEGA
COMPA&#209;EROS SEXUALES: 1
ITS: NIEGA 
GRUPO SANGUINEO: A+
CONTROLES PRENALES: 6

PACIENTE NO TRAE PARACLINICOS 

ECOGRAFIA 
14/1/19 EMB 25.3 SEMANAS PARA HOY 39.6 SEMANAS 
5/9/18 EMB 6.1 SEMANA S PARA HOY 39.2 SEMANAS * 7-&a

In [49]:
# read paraclinics. Assume that the date is just bmonthre them, or in the same line

pos = ['pos', '\+', 'reac']
neg = ['neg', '-', 'no']
vih1  = 'vih\s*[1i]'
vih2 = 'vih\s*(:?2|ii)'
vih12 = 'vih\s*[1i]\s*(:?2|ii)'
prt = '(:?PRUEBA RAPIDA TREPONEMICA|PRT)'
vdrl = '(:?VDRL|SIF[A-Z]*|RPR|FTA|FTA ABS)'
floatParse = '[0-9]*[\.,]?[0-9]+'

hematology = {
'HCT' : '(HCT|HTO|HTC|HEMATO[A-Z]*)',
'HB' : '(HB|HEMOGLOB[A-Z]*)',
'LEU' : '(LEU[A-Z]*)',
'NEU' : '(NEU[A-Z]*)',
'LIN' : '(LIN)',
'MONO' : '(MONO[A-Z]*)',
'PLAQ' : '(PL[A-Z]*|PQT)'}

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
sep= '\s*[,;:]?\s*'
separadorFecha = '(?:[\.\\/-]|DE|DEL|\s)'
date =  ' \(?' +  '((?:[0-9]|[0-3][0-9])'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

def parseParaclinics(r, lastDate = None):
    
    if not isinstance(r, str):
        rET = ET.fromstring(r.RegistroXML)
        rTxt = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', rET))
        rTxt = parsingDatabaseUtils.removeWords(rTxt, ['de', 'y', 'a', 'el', 'los'])
    else:
        rTxt = r
        
        
    lastDate = None
    results = collections.defaultdict(dict)
    results['text'] = rTxt
    if re.findall('(no|sin|ni) (prese[a-z]*|tien[a-z]*|tra[a-z]*)(\s)*para', rTxt):
        results['sinParaclinicos'] = True
        return results
    
    for i, l in enumerate(rTxt.splitlines()):
        l = ' ' + l
        # use instead dateparser.search.search_dates  <- it goes too slow
        d = re.findall(date, l, re.IGNORECASE)
        # dFiltered = [dateparser.parse(dd) for dd in d]
        #dFiltered = [dd for dd in dFiltered if dd]

        if len(d) == 1.:
            dateParsed = parsingDatabaseUtils.parseDate(d[0])
            if dateParsed:
                #lastDate ='%02d/%02d/%d' %(dateParsed.day, dateParsed.month, dateParsed.year)
                lastDate = '/'.join(dateParsed)
        elif len(d):
            lastDate = None
            
            
        if len(d) <= 1 and lastDate:
            #Hematology
            for p, v in hematology.items():
                hematologyRes = re.findall('%s (%s)[^x]' % (v, floatParse), l, re.IGNORECASE)
                if hematologyRes:
                    results[lastDate][p] = hematologyRes[0][1]

            #sifilis
            sif1 = ' ' + vdrl + " (%s)" % '|'.join(pos + neg)
            searchSif1 = re.findall(sif1, l, re.IGNORECASE)
            sif2 = ' ' + prt + " (%s)" % '|'.join(pos + neg)    
            searchSif2 = re.findall(sif2, l, re.IGNORECASE)
            if searchSif1:
                results[lastDate]['vdrl'] =searchSif1[0][1] in pos
            if searchSif2:
                results[lastDate]['prt'] =searchSif2[0][1] in pos
            # TODO: vih
    return results

In [41]:
def paraclinicsToDF(p):
    res = {}
    res['noParaclinicalTestsConfirmed'] = 'sinParaclinicos' in p
    i = 0
    for date in sorted(p.keys()):
        if date  in ['sinParaclinicos', 'text']: 
            continue
        res['day_%d' % i] = date
        for k, v in p[date].items():
            res['day_%d' % i + k] = v 
        i += 1
    return res

In [42]:
import cProfile

In [43]:
paraClinics = {}
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if p.epicrisis is None:
        continue
    d =parseParaclinics(p.epicrisis)
    paraClinics[c] = paraclinicsToDF(parseParaclinics(p.epicrisis))


In [46]:
dfParaClinics.noParaclinicalTestsConfirmed.value_counts()

False    1822
True       95
Name: noParaclinicalTestsConfirmed, dtype: int64

In [47]:
dfParaClinics = pandas.DataFrame.from_dict(paraClinics, orient = 'index')
dfParaClinics = dfParaClinics[sorted(dfParaClinics.columns)]
dfParaClinics.to_csv('paraclinics.csv')

In [68]:
for c, p in paraClinics.items():
    if len(p) != 1:
        print(paraClinics[c]['text'])
        print('-----------------')
        for k in p:
            if k != 'text':
                print(k, p[k])
        print('-----------------')

In [ ]:
def merge2DF(df1, df2):
    """
    Returns the merge of 2 df with the same index (1 to 1)
    In case of columns with the same name, check if the values are the same (or one of them is NA)
    Creates new error column to see where there has been problems
    """
    pass